# Setup

In [1]:
import pandas as pd
from tqdm import tqdm
import json
import numpy as np
import matplotlib.pyplot as plt

from random import shuffle

from functools import partial
from sklearn.metrics import classification_report as sklcls_report
from sklearn.metrics import f1_score, accuracy_score
from transformers import DataCollatorForLanguageModeling, AutoTokenizer, TFAutoModelForCausalLM, AdamWeightDecay
from os.path import join, exists

In [2]:
from lib.BBData import character_dict, model_name, random_state
from lib.BBDataLoad import dialogpt_preprocess_function, load_char_df, get_chatbot_predictions, merge_df_for_metrics
from random import sample

from lib.wip.frequency import sentence_preprocess, FrequencyChatbotClassifier
from sklearn.metrics import confusion_matrix

In [80]:
TEST_WITH_BATCH = False
PLOT_SCORE_BATCH = False
TEST_CHATBOT = False
SAVE_CHATBOT_PREDICTION = False

In [3]:
characters = list(character_dict.keys())
classification_report = partial(sklcls_report, target_names=characters)
# characters.remove('Default')

mass_value = 0.3

In [4]:
# Mount google drive, if in Colaboratory environment
import os
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    base_folder = '/content/drive/My Drive/unibo/NLP_project/BarneyBot'
    os.system("pip install datasets")
    os.system("pip install transformers")
    os.system("pip install rouge_score")
    os.system("pip install -U sentence-transformers")
else:
    # base_folder = os.getcwd()
    base_folder = '..'

out_folder = os.path.join(base_folder, 'Data', 'Characters')

# Data Loading

In [5]:
character_docs = dict()
class_docs = [[], []]
for character in characters:
    if character == 'Default':
        df = pd.read_csv(os.path.join(out_folder, character, f'{character}.tsv'), 
                         names=[character, 'response'], sep='\t')
        df['response'] = df['response'].apply(lambda x: x[3:])
        df[character] = df[character].apply(lambda x: x[3:])
    else:
        df = pd.read_csv(os.path.join(out_folder, character, f'{character}.csv'))
    tmp_list = df['response'].tolist()
    character_docs[character] = tmp_list
    class_docs[0] += tmp_list
    class_docs[1] += [character for _ in tmp_list]

In [6]:
for character in characters:
    for i in tqdm(range(len(character_docs[character]))):
        sentence, relevant = sentence_preprocess(character_docs[character][i])
        if relevant:
            character_docs[character][i] = sentence

100%|██████████| 737332/737332 [02:39<00:00, 4618.10it/s]


In [7]:
test_size = 0.33
character_docs_train = {}
character_docs_test = {}
n_docs = 0
for c in characters:
    shuffle(character_docs[c])
    end_idx = int(len(character_docs[c]) * test_size)
    character_docs_train[c] = character_docs[c][end_idx:]
    n_docs += len(character_docs[c][end_idx:])
    character_docs_test[c] = character_docs[c][:end_idx]

# Word Frequency

Loading word frequencies

In [8]:
"""wordfreqs_train = dict()
for character in tqdm(characters):
    wordfreqs_train[character] = get_word_frequency(' '.join(character_docs_train[character]), f_sorted=True)

wordfreqs_test = dict()
for character in tqdm(characters):
    wordfreqs_test[character] = get_word_frequency(' '.join(character_docs_test[character]), f_sorted=True)"""

"wordfreqs_train = dict()\nfor character in tqdm(characters):\n    wordfreqs_train[character] = get_word_frequency(' '.join(character_docs_train[character]), f_sorted=True)\n\nwordfreqs_test = dict()\nfor character in tqdm(characters):\n    wordfreqs_test[character] = get_word_frequency(' '.join(character_docs_test[character]), f_sorted=True)"

In [9]:
"""wordfreqs_reduced_train = dict()
for character in characters:
    wordfreqs_reduced_train[character] = filter_by_weights(wordfreqs_train[character], mass=0.3)

wordfreqs_reduced_test = dict()
for character in characters:
    wordfreqs_reduced_test[character] = filter_by_weights(wordfreqs_test[character], mass=0.3)"""

'wordfreqs_reduced_train = dict()\nfor character in characters:\n    wordfreqs_reduced_train[character] = filter_by_weights(wordfreqs_train[character], mass=0.3)\n\nwordfreqs_reduced_test = dict()\nfor character in characters:\n    wordfreqs_reduced_test[character] = filter_by_weights(wordfreqs_test[character], mass=0.3)'

Loading TF-IDF matrix

In [10]:
"""tfidf_vectorizer = TfidfVectorizer(input='content', stop_words='english')"""

"tfidf_vectorizer = TfidfVectorizer(input='content', stop_words='english')"

In [11]:
"""tfidfs_train = get_tfidfs([' '.join(character_docs_train[character]) for character in characters], characters, tfidf_vectorizer)"""

"tfidfs_train = get_tfidfs([' '.join(character_docs_train[character]) for character in characters], characters, tfidf_vectorizer)"

In [12]:
"""tfidfs_reduced_train = dict()
for character in characters:
    tfidfs_reduced_train[character] = filter_by_weights(tfidfs_train[character], mass=mass_value)"""

'tfidfs_reduced_train = dict()\nfor character in characters:\n    tfidfs_reduced_train[character] = filter_by_weights(tfidfs_train[character], mass=mass_value)'

# WordCloud Plot (Example)

In [13]:
"""from lib.BBVisualizations import BBVisualization
BBVisualization.load_visualization("wordcloud").plot(freqdict=tfidfs_reduced_train['Barney'])"""

'from lib.BBVisualizations import BBVisualization\nBBVisualization.load_visualization("wordcloud").plot(freqdict=tfidfs_reduced_train[\'Barney\'])'

# Frequency Pairwise Similarity (Example)

In [14]:
"""freq_pairwise_sim(wordfreqs_reduced_train['Fry'], wordfreqs_reduced_test['Fry'])"""

"freq_pairwise_sim(wordfreqs_reduced_train['Fry'], wordfreqs_reduced_test['Fry'])"

# Test Classifier Performances Against Test Set

### Word Frequency

In [55]:
tfidf_classifier = FrequencyChatbotClassifier(characters, mode='word frequency')
tfidf_classifier.train(list(character_docs_train.values()))

y_true = range(0, len(characters))
y_pred = []
raw_predictions = {c:[] for c in characters}
for c in tqdm(range(len(characters))):
    prediction = tfidf_classifier.predict(character_docs_test[characters[c]])
    raw_predictions[characters[c]].append(prediction)
    y_pred.append(np.argmax(list(prediction.values())))

print('WF classifier test accuracy: {:.2f}'.format(
             sum([y_pred[i] == y_true[i] for i in range(len(y_true))]) / len(y_true)))
confusion_matrix(y_true, y_pred)

100%|██████████| 9/9 [00:01<00:00,  4.78it/s]

WF classifier test accuracy: 1.00


array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int64)

### TFIDF

In [58]:
tfidf_classifier = FrequencyChatbotClassifier(characters, mode='tf-idf')
tfidf_classifier.train(list(character_docs_train.values()))

y_true = range(0, len(characters))
y_pred = []
raw_predictions = {c:[] for c in characters}
for c in tqdm(range(len(characters))):
    prediction = tfidf_classifier.predict(character_docs_test[characters[c]])
    raw_predictions[characters[c]].append(prediction)
    y_pred.append(np.argmax(list(prediction.values())))

print('TF-IDF classifier test accuracy: {:.2f}'.format(
             sum([y_pred[i] == y_true[i] for i in range(len(y_true))]) / len(y_true)))
confusion_matrix(y_true, y_pred)

100%|██████████| 9/9 [00:01<00:00,  4.72it/s]

TF-IDF classifier test accuracy: 1.00


array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int64)

### c-TF-IDF 

In [25]:
ctfidf_classifier = FrequencyChatbotClassifier(characters, mode='c-tf-idf', )
ctfidf_classifier.train(list(character_docs_train.values()), n_tot_docs=n_docs)

y_true = range(0, len(characters))
y_pred = []
raw_predictions = {c:[] for c in characters}
for c in tqdm(range(len(characters))):
    prediction = ctfidf_classifier.predict(character_docs_test[characters[c]])
    raw_predictions[characters[c]].append(prediction)
    y_pred.append(np.argmax(list(prediction.values())))

print('c-TF-IDF classifier test accuracy: {:.2f}'.format(
             sum([y_pred[i] == y_true[i] for i in range(len(y_true))]) / len(y_true)))
confusion_matrix(y_true, y_pred)

100%|██████████| 9/9 [00:01<00:00,  4.69it/s]

c-TF-IDF classifier test accuracy: 1.00


array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int64)

# Test over a batch of sentences

In [75]:
if TEST_WITH_BATCH:
    batch = 3
    complex_docs_test = []
    y_true = []
    for c in characters:
        tmp_list = sample(character_docs_test[c], 50)
        complex_docs_test += [
            tmp_list[d:d+batch] for d in range(0, len(tmp_list), batch)
            ]
        y_true += [characters.index(c) for _ in range(round(len(tmp_list)/batch))]

In [74]:
if TEST_WITH_BATCH:
    wf_classifier = FrequencyChatbotClassifier(characters, mode='word frequency')
    wf_classifier.train(list(character_docs_train.values()))

    y_pred = []
    for c in tqdm(range(len(complex_docs_test))):
        prediction = wf_classifier.predict(complex_docs_test[c])
        y_pred.append(np.argmax(list(prediction.values())))
    print(classification_report(y_true, y_pred))
    confusion_matrix(y_true, y_pred)

In [76]:
if TEST_WITH_BATCH:
    tfidf_classifier = FrequencyChatbotClassifier(characters, mode='tf-idf')
    tfidf_classifier.train(list(character_docs_train.values()))

    y_pred = []
    for c in tqdm(range(len(complex_docs_test))):
        prediction = tfidf_classifier.predict(complex_docs_test[c])
        y_pred.append(np.argmax(list(prediction.values())))
    print(classification_report(y_true, y_pred))
    confusion_matrix(y_true, y_pred)

In [78]:
if TEST_WITH_BATCH:
    ctfidf_classifier = FrequencyChatbotClassifier(characters, mode='c-tf-idf')
    ctfidf_classifier.train(list(character_docs_train.values()), n_tot_docs=n_docs)

    y_pred = []
    for c in tqdm(range(len(complex_docs_test))):
        prediction = ctfidf_classifier.predict(complex_docs_test[c])
        y_pred.append(np.argmax(list(prediction.values())))
    print(classification_report(y_true, y_pred))
    confusion_matrix(y_true, y_pred)

#### Experimenting the accuracy over different batch dimensions

In [68]:
if PLOT_SCORE_BATCH:
    def run_experiment(b: int, model: FrequencyChatbotClassifier, c='Barney', fix_dataset_size=None):
        if fix_dataset_size == None:
            tmp_list = character_docs_test[c]
        else:
            tmp_list = sample(character_docs_test[c], fix_dataset_size)
        complex_docs_test = [tmp_list[d:d+b] for d in range(0, len(tmp_list), b)]
        y_true = [characters.index(c) for _ in complex_docs_test]

        y_pred = []
        for doc in range(len(complex_docs_test)):
            prediction = model.predict(complex_docs_test[doc])
            y_pred.append(np.argmax(list(prediction.values())))
        # score = sum([y_pred[i] == y_true[i] for i in range(len(y_true))]) / len(y_true)
        score = accuracy_score(y_true, y_pred)
        # score = f1_score(y_true, y_pred, average='macro')
        return score

In [69]:
if PLOT_SCORE_BATCH:
    character = 'Barney'
    max_batch = 50+1
    batches = range(1, max_batch, 1)
    test_sample_size = None

    wf_classifier = FrequencyChatbotClassifier(characters, mode='word frequency')
    wf_classifier.train(list(character_docs_train.values()))
    score = [run_experiment(b, wf_classifier, c=character, fix_dataset_size=test_sample_size) for b in tqdm(batches)]
    plt.plot(batches, score, label='wf')

    tfidf_classifier = FrequencyChatbotClassifier(characters, mode='tf-idf')
    tfidf_classifier.train(list(character_docs_train.values()))
    score = [run_experiment(b, tfidf_classifier, c=character, fix_dataset_size=test_sample_size) for b in tqdm(batches)]
    plt.plot(batches, score, label='tf-idf')

    ctfidf_classifier = FrequencyChatbotClassifier(characters, mode='c-tf-idf')
    ctfidf_classifier.train(list(character_docs_train.values()), n_tot_docs=n_docs)
    score = [run_experiment(b, ctfidf_classifier, c=character, fix_dataset_size=test_sample_size) for b in tqdm(batches)]
    plt.plot(batches, score, label='c-tf-idf')

    plt.xlabel('Batch size')
    plt.ylabel('f1 score')
    plt.title(character)
    plt.legend()
    plt.show()

In [70]:
if PLOT_SCORE_BATCH:
    character = 'Sheldon'
    max_batch = 50+1
    batches = range(1, max_batch, 1)
    test_sample_size = None

    wf_classifier = FrequencyChatbotClassifier(characters, mode='word frequency')
    wf_classifier.train(list(character_docs_train.values()))
    score = [run_experiment(b, wf_classifier, c=character, fix_dataset_size=test_sample_size) for b in tqdm(batches)]
    plt.plot(batches, score, label='wf')

    tfidf_classifier = FrequencyChatbotClassifier(characters, mode='tf-idf')
    tfidf_classifier.train(list(character_docs_train.values()))
    score = [run_experiment(b, tfidf_classifier, c=character, fix_dataset_size=test_sample_size) for b in tqdm(batches)]
    plt.plot(batches, score, label='tf-idf')

    ctfidf_classifier = FrequencyChatbotClassifier(characters, mode='c-tf-idf')
    ctfidf_classifier.train(list(character_docs_train.values()), n_tot_docs=n_docs)
    score = [run_experiment(b, ctfidf_classifier, c=character, fix_dataset_size=test_sample_size) for b in tqdm(batches)]
    plt.plot(batches, score, label='c-tf-idf')

    plt.xlabel('Batch size')
    plt.ylabel('f1 score')
    plt.title(character)
    plt.legend()
    plt.show()

In [71]:
if PLOT_SCORE_BATCH:
    character = 'Vader'
    max_batch = 50+1
    batches = range(1, max_batch, 1)
    test_sample_size = None

    wf_classifier = FrequencyChatbotClassifier(characters, mode='word frequency')
    wf_classifier.train(list(character_docs_train.values()))
    score = [run_experiment(b, wf_classifier, c=character, fix_dataset_size=test_sample_size) for b in tqdm(batches)]
    plt.plot(batches, score, label='wf')

    tfidf_classifier = FrequencyChatbotClassifier(characters, mode='tf-idf')
    tfidf_classifier.train(list(character_docs_train.values()))
    score = [run_experiment(b, tfidf_classifier, c=character, fix_dataset_size=test_sample_size) for b in tqdm(batches)]
    plt.plot(batches, score, label='tf-idf')

    ctfidf_classifier = FrequencyChatbotClassifier(characters, mode='c-tf-idf')
    ctfidf_classifier.train(list(character_docs_train.values()), n_tot_docs=n_docs)
    score = [run_experiment(b, ctfidf_classifier, c=character, fix_dataset_size=test_sample_size) for b in tqdm(batches)]
    plt.plot(batches, score, label='c-tf-idf')

    plt.xlabel('Batch size')
    plt.ylabel('f1 score')
    plt.title(character)
    plt.legend()
    plt.show()

# Test Classifiers On Chatbots (Not Cached!)

In [25]:
tfidf_classifier = FrequencyChatbotClassifier(characters, mode='c-tf-idf')
tfidf_classifier.train(list(character_docs.values()), n_tot_docs=len(list(character_docs.values())))

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=join("..", "cache"))
tokenizer.pad_token = '#'
data_collator = DataCollatorForLanguageModeling(mlm=False, tokenizer=tokenizer, return_tensors='tf')

In [12]:
characters_noDefault = characters.copy()
characters_noDefault.remove('Default')

In [41]:
### create dataset
n_tests = 15
test_size = 10
selected_indeces = [i for i in range(test_size)]
doc_test = []
batch_size = 8
override_predictions = True
predictions = {c:[] for c in characters}
raw_predictions = {c:[] for c in characters}
msw_predictions = {c:[] for c in characters}
print('Creating dataset...')
if n_tests > 1 and not override_predictions:
    raise Exception('must override previous predictions if you need more tests')

for i in range(n_tests):
    print(f'Run {i}/{n_tests}')
    for character in tqdm(characters_noDefault):
        character_checkpoint = join(out_folder, character, character_dict[character]['checkpoint_folder'])
        model_chatbot = TFAutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=character_checkpoint) if override_predictions else None
        # if model_chatbot:
        #     model_chatbot.compile(optimizer=AdamWeightDecay(learning_rate=2e-5))
        character_hg = load_char_df(character, base_folder)
        # This transform in a sequence of tokens ours dataset
        tokenized_character_hg = character_hg.map(lambda row: dialogpt_preprocess_function(row, tokenizer), batched=False)

        # Define tensorflow datasets
        encoded_test_set = tokenized_character_hg["test"].to_tf_dataset(
            columns=["input_ids", "attention_mask", "labels"],
            shuffle=False,
            batch_size=batch_size,
            collate_fn=data_collator,
        )

        # Takes a subset of the testset as sample question
        character_hg['test'] = character_hg['test'].shuffle(random_state).select(selected_indeces)
        sample_questions = character_hg['test']['context/0']
        # Sampling generation method
        predictions_sampling = get_chatbot_predictions(
            sample_questions,
            model_chatbot,
            character_dict[character]['prediction_filename'] + '_sampling.json',
            "Sampling",
            character,
            tokenizer,
            base_folder,
            override_predictions=override_predictions
        )
                                                    
        sentences = merge_df_for_metrics(character_hg['test'], None, None, predictions_sampling, tokenizer)['prd_sampling'].tolist()
        sentences = [sentence_preprocess(s) for s in sentences]
        doc_test.append([s[0] for s in sentences if s[1]])
        

Creating dataset...
Run 0/15


  0%|          | 0/8 [00:00<?, ?it/s]All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ..\Data\Characters\Barney\barney_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Using custom data configuration default-d78476db32c8db98
Reusing dataset csv (cache\csv\default-d78476db32c8db98\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
100%|██████████| 1/1 [00:00<00:00, 83.34it/s]
Loading cached split indices for dataset at cache\csv\default-d78476db32c8db98\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-651037d43e5e3dbd.arrow and cache\csv\default-d78476db32c8db98\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-7e4903d85e508b43.arrow
Loading cached split indices for dataset at cache\csv\de

Creating predictions


 12%|█▎        | 1/8 [00:44<05:11, 44.49s/it]All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ..\Data\Characters\Sheldon\sheldon_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Using custom data configuration default-8207271dc09e0a46
Reusing dataset csv (cache\csv\default-8207271dc09e0a46\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
100%|██████████| 1/1 [00:00<00:00, 142.87it/s]
Loading cached split indices for dataset at cache\csv\default-8207271dc09e0a46\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-46b6cd7566c1d243.arrow and cache\csv\default-8207271dc09e0a46\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-e3e40e36f25dded7.arrow
Loading cached split indices for dataset at c

Creating predictions


...

 75%|███████▌  | 6/8 [02:14<00:45, 22.72s/it]All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ..\Data\Characters\Joey\joey_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Using custom data configuration default-b81141adc87b87bf
Reusing dataset csv (cache\csv\default-b81141adc87b87bf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
100%|██████████| 1/1 [00:00<00:00, 500.04it/s]
Loading cached split indices for dataset at cache\csv\default-b81141adc87b87bf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-e2d3926d0ffd0fd3.arrow and cache\csv\default-b81141adc87b87bf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-f888d4a055d9f5e2.arrow
Loading cached split indices for dataset at cache\c

Creating predictions


 88%|████████▊ | 7/8 [03:14<00:34, 34.85s/it]All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ..\Data\Characters\Phoebe\phoebe_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Using custom data configuration default-48cc5f719001fd3a
Reusing dataset csv (cache\csv\default-48cc5f719001fd3a\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
100%|██████████| 1/1 [00:00<00:00, 124.96it/s]
Loading cached split indices for dataset at cache\csv\default-48cc5f719001fd3a\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-16c140b5da191e7c.arrow and cache\csv\default-48cc5f719001fd3a\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-ed121df269d87298.arrow
Loading cached split indices for dataset at cac

Creating predictions


100%|██████████| 8/8 [03:39<00:00, 27.50s/it]


In [59]:
y_true = []
for i in range(len(doc_test)):
    y_true += [i % len(characters_noDefault)]

In [63]:
### prediction on last test
print('Classification...')
y_pred = []
raw_predictions = {c:[] for c in characters}
for d in tqdm(doc_test):
    prediction = wf_classifier.predict(d)
    raw_predictions[characters[c]].append(prediction)
    y_pred.append(np.argmax(list(prediction.values())))

print('TF-IDF classifier test accuracy: {:.2f}'.format(accuracy_score(y_true, y_pred)))
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

Classification...


100%|██████████| 120/120 [00:00<00:00, 203.04it/s]

TF-IDF classifier test accuracy: 0.64
              precision    recall  f1-score   support

      Barney       0.79      0.73      0.76        15
     Sheldon       0.88      1.00      0.94        15
       Harry       0.80      0.53      0.64        15
         Fry       0.62      0.33      0.43        15
      Bender       0.45      0.60      0.51        15
       Vader       1.00      0.73      0.85        15
        Joey       0.55      0.40      0.46        15
      Phoebe       0.46      0.80      0.59        15
     Default       0.00      0.00      0.00         0

    accuracy                           0.64       120
   macro avg       0.62      0.57      0.58       120
weighted avg       0.69      0.64      0.65       120




c:\Users\User\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[11,  0,  0,  0,  0,  0,  1,  3,  0],
       [ 0, 15,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  8,  1,  3,  0,  2,  0,  0],
       [ 1,  0,  0,  5,  7,  0,  0,  1,  1],
       [ 1,  0,  0,  2,  9,  0,  1,  1,  1],
       [ 0,  0,  1,  0,  1, 11,  0,  1,  1],
       [ 0,  1,  0,  0,  0,  0,  6,  8,  0],
       [ 0,  1,  1,  0,  0,  0,  1, 12,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0]], dtype=int64)

In [62]:
### prediction on last test
print('Classification...')
y_pred = []
raw_predictions = {c:[] for c in characters}
for d in tqdm(doc_test):
    prediction = tfidf_classifier.predict(d)
    raw_predictions[characters[c]].append(prediction)
    y_pred.append(np.argmax(list(prediction.values())))

print('TF-IDF classifier test accuracy: {:.2f}'.format(accuracy_score(y_true, y_pred)))
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

Classification...


100%|██████████| 120/120 [00:01<00:00, 108.05it/s]

TF-IDF classifier test accuracy: 0.69
              precision    recall  f1-score   support

      Barney       1.00      0.80      0.89        15
     Sheldon       0.83      1.00      0.91        15
       Harry       1.00      0.67      0.80        15
         Fry       0.62      0.33      0.43        15
      Bender       0.45      0.67      0.54        15
       Vader       1.00      0.93      0.97        15
        Joey       0.56      0.33      0.42        15
      Phoebe       0.50      0.80      0.62        15
     Default       0.00      0.00      0.00         0

    accuracy                           0.69       120
   macro avg       0.66      0.61      0.62       120
weighted avg       0.75      0.69      0.70       120




c:\Users\User\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[12,  0,  0,  0,  0,  0,  1,  2,  0],
       [ 0, 15,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 10,  0,  2,  0,  1,  0,  2],
       [ 0,  0,  0,  5,  8,  0,  0,  1,  1],
       [ 0,  0,  0,  3, 10,  0,  1,  1,  0],
       [ 0,  0,  0,  0,  1, 14,  0,  0,  0],
       [ 0,  2,  0,  0,  0,  0,  5,  8,  0],
       [ 0,  1,  0,  0,  1,  0,  1, 12,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0]], dtype=int64)

In [64]:
### prediction on last test
print('Classification...')
y_pred = []
raw_predictions = {c:[] for c in characters}
for d in tqdm(doc_test):
    prediction = ctfidf_classifier.predict(d)
    raw_predictions[characters[c]].append(prediction)
    y_pred.append(np.argmax(list(prediction.values())))

print('TF-IDF classifier test accuracy: {:.2f}'.format(accuracy_score(y_true, y_pred)))
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

Classification...


100%|██████████| 120/120 [00:00<00:00, 209.43it/s]

TF-IDF classifier test accuracy: 0.72
              precision    recall  f1-score   support

      Barney       1.00      0.80      0.89        15
     Sheldon       0.88      1.00      0.94        15
       Harry       1.00      0.87      0.93        15
         Fry       0.62      0.33      0.43        15
      Bender       0.50      0.67      0.57        15
       Vader       1.00      1.00      1.00        15
        Joey       0.43      0.40      0.41        15
      Phoebe       0.55      0.73      0.63        15
     Default       0.00      0.00      0.00         0

    accuracy                           0.73       120
   macro avg       0.67      0.64      0.64       120
weighted avg       0.75      0.72      0.73       120




c:\Users\User\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[12,  0,  0,  0,  0,  0,  2,  1,  0],
       [ 0, 15,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 13,  0,  1,  0,  1,  0,  0],
       [ 0,  0,  0,  5,  9,  0,  1,  0,  0],
       [ 0,  0,  0,  3, 10,  0,  1,  0,  1],
       [ 0,  0,  0,  0,  0, 15,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  6,  8,  0],
       [ 0,  1,  0,  0,  0,  0,  3, 11,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0]], dtype=int64)

In [81]:
### save predictions
if SAVE_CHATBOT_PREDICTION:
    append_predictions = True
    override_predictions = False
    predictions_file = join('..', 'metrics', 'tfidf_predictions.json')

    if append_predictions and exists(predictions_file):
        with open(predictions_file, 'r', encoding='utf-8') as file:
            predictions_dict = json.load(file)
    elif override_predictions or not exists(predictions_file):
        predictions_dict = {'one_hot':{c:[] for c in characters}, 'raw_predictions': {c:[] for c in characters}}
    else:
        raise NotImplementedError
        
    for c in characters_noDefault:
        predictions_dict['one_hot'][c] += predictions[c]
        predictions_dict['raw_predictions'][c] += raw_predictions[c]

    with open(predictions_file, 'w', encoding='utf-8') as file:
        json.dump(predictions_dict, file)